# Aufbau der Notebooks [Philipp]

- Multivalue bei Notebooks angucken
- Wie installiere ich den "scheiß"
- Widgets: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html


In [2]:
from enum import Enum
import ipywidgets as widgets
from pandas import read_csv

In [3]:
# Globals

PATH_RAW_DATA = "./raw/"
PATH_FEATURE_DATA = "./features/"
PATH_EXPLORATION_DATA = "./exploration/"
PATH_MODEL = "./model/"
DATA_SOURCE_KIDAQ = ["TEST_NAME", "TEST_TYPE", "RPM", "FLOW_RATE", "P1", "P2"]

DATA_SOURCE_VID = [
    "TEST_NAME",
    "TEST_TYPE",
    "RPM",
    "FLOW_RATE",
    "S1",
    "S2",
    "S3",
    "S4",
    "S5",
    "S6",
    "S7",
    "S8",
]
FEATURE = [
    "STD",
    "RANGE",
    "IQR",
    "MEAN_MEDIAN",
    "FFT",
]
OPERATING_POINT_FREQ = ["RPM725", "RPM1450", "RPM2175", "RPM2900"]
OPERATING_POINT_FLOW_RATE = ["PCT0", "PCT25", "PCT50", "PCT75", "PCT100"]

LEARNER = Enum("LEARNER", ["DNN", "DT"])

DEFAULT_LEARNER = LEARNER.DNN
DEFAULT_RAW_DATA = DATA_SOURCE_KIDAQ
print(widgets.__version__)
tags = widgets.TagsInput(
    value=DEFAULT_RAW_DATA, allowed_tags=DEFAULT_RAW_DATA, allow_duplicates=False
)
tags

8.0.6


TagsInput(value=['TEST_NAME', 'TEST_TYPE', 'RPM', 'FLOW_RATE', 'P1', 'P2'], allow_duplicates=False, allowed_ta…

## 1. Aufgabe und Daten erklären/beschreiben [Philipp]


## 3. Preprocessing [Valerij]

Aufgeteilt nach KIDAQ und VIB (separat um es einfach zu halten)

- Auswahl der Fenstergröße in Millisekunden
- Auswahl der Abtastrate
- Multi-Selektion der Aggregation (avg, mean, std, ...)
- Frequenzanalyse
- Fourier-Transformation
- Fenstergröße nach Frequenzbereichen

### 3.1 Vorbereitung der Tainings- und Testdaten

Multi-Selektion für:

- Features
- Betriebspunkte (RPM/FLOWRATE)
- Klassifikationsarten (Szenario / Testdurchlauf)


## 4. Deskriptive/Explorative Datenanalyse [Philipp]

- Plots
- Beschreibung der Plots


## 5. Machine Learning [Kevin]

Multi-Selektion für:

- Auswahl der Featuredateien (Train/Testdaten)
- Auswahl des Learners
- Konfiguration der Hyperparameter
- Live-Validation des Models mit vorausgewählten Testdaten (Random-Search, ...)


In [3]:
FEATURE_TRAIN_FILE = "train.csv"
FEATURE_TEST_FILE = "test.csv"

DEFAULT_OPERATING_POINTS = OPERATING_POINT_FREQ + OPERATING_POINT_FLOW_RATE


# DNN
DNN_EXPLORATION_TARGET_VAL_ACCURACY = 0.9
DNN_EXPLORATION_MAX_ITER = 100
DNN_EXPLORATION_HIDDEN_LAYERS_MIN = 2
DNN_EXPLORATION_HIDDEN_LAYERS_MAX = 4
DNN_EXPLORATION_NEURONS_MIN = 8
DNN_EXPLORATION_NEURONS_MAX = 64


DNN_EARLY_STOPPING_PATIENCE = 50
DNN_VERBOSE = 0
DNN_EPOCHS = 100
DNN_BATCH_SIZE = 32
DNN_BATCH_NORMALIZATION = True


# DT
DT_MAX_DEPTH = 6
DT_NUM_ESTIMATORS = 100

In [ ]:
train_data = read_csv(FEATURE_TRAIN_FILE, header=None, delimiter=";")
test_data = read_csv(FEATURE_TEST_FILE, header=None, delimiter=";")


train_x, train_y = train_data[:, 2:], train_data[:, 1:2]
test_x, test_y = test_data[:, 2:], test_data[:, 1:2]

In [ ]:
# DT
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

labelEncoder = LabelEncoder()
labelEncoder = labelEncoder.fit(train_y)
label_encoded_train_y = labelEncoder.transform(train_y)
label_encoded_test_y = labelEncoder.transform(test_y)


xgb = XGBClassifier(
    tree_method="hist",
    enable_categorical=True,
    max_depth=DT_MAX_DEPTH,
    n_estimators=DT_NUM_ESTIMATORS,
)
# fit model
xgb.fit(train_x, label_encoded_train_y, eval_set=[(test_x, label_encoded_test_y)])


preds = xgb.predict(test_x)
accuracy = accuracy_score(label_encoded_test_y, preds)

xgb.save_model(PATH_MODEL)

In [ ]:
# DNN
import random
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, InputLayer, BatchNormalization
from tensorflow.python.keras.callbacks import EarlyStopping

onehotencoder = OneHotEncoder()
onehotencoder = onehotencoder.fit(train_y)
onehot_encoded_train_y = onehotencoder.transform(train_y).toarray()
onehot_encoded_test_y = onehotencoder.transform(test_y).toarray()

exploration_results = []


n_features = train_x.shape[1]
categories = len(onehot_encoded_train_y[0])

test_acc = 0.0
model = Sequential()
interation = 0
while (
    test_acc < DNN_EXPLORATION_TARGET_VAL_ACCURACY
    and interation < DNN_EXPLORATION_MAX_ITER
):
    model.add(InputLayer(input_shape=(n_features,)))
    model.add(BatchNormalization())
    dense_count = random.randint(
        DNN_EXPLORATION_HIDDEN_LAYERS_MIN, DNN_EXPLORATION_HIDDEN_LAYERS_MAX
    )
    dense_neurons = []
    for i in range(0, dense_count):
        neurons = random.randint(
            DNN_EXPLORATION_NEURONS_MIN, DNN_EXPLORATION_NEURONS_MAX
        )
        dense_neurons.append(neurons)
        model.add(Dense(neurons, activation="tanh"))
        model.add(BatchNormalization())
    model.add(Dense(categories, activation="sigmoid"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    model.fit(
        train_x,
        onehot_encoded_train_y,
        epochs=DNN_EPOCHS,
        batch_size=DNN_BATCH_SIZE,
        verbose=1,
        validation_data=(test_x, onehot_encoded_test_y),
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                patience=DNN_EARLY_STOPPING_PATIENCE,
                restore_best_weights=True,
            )
        ],
    )

    train_loss, train_acc = model.evaluate(train_x, onehot_encoded_train_y)
    test_loss, test_acc = model.evaluate(test_x, onehot_encoded_test_y)

    exploration_results.append(
        {
            "dense_count": dense_count,
            "dense_neurons": dense_neurons,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
        }
    )
    interation += 1
    model = Sequential()

model.save(PATH_MODEL)

## 6. Modelanalyse des Learners [Kevin]

- Vorherige Auswahl eines Learners
- Feature Importance
- Korrelationsmatrix
- Konfusionsmatrix
- Post-Validation des Models mit auswählbaren Daten


## 7. Statische Interpretation des Resultats

- Welches Ergebnis haben wir erzielt und wie kann man es anwenden?
